# Depression Classification from texts using BERT

Utilizing BERT networks for binary text classification. Each individual message or utterance within these dialogues is associated with one of two labels: "depressed" or "not depressed."

## Workflow:
1. Import Data
2. Data Preprocessing
3. BERT Model Building
4. Training and Validation
5. Model Evalutaion and Testing
6. Saving The Model

**Use google colab for a free GPU**

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.4 MB/s eta 0:00:00


# Importing Librabries

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Depression Dataset/Depression_Detection.csv')


In [ ]:
import nltk
import torch
from transformers import BertTokenizer, BertForSequenceClassification
nltk.download('punkt')
import re
!pip install autocorrect
from autocorrect import Speller
!pip install unidecode
import unidecode
!pip install contractions
import contractions
from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=dcc8e4f1ac3e52125a849e35739c930124c2ea80df45f582297383e92790473e
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.0 MB/s eta 0:00:00


# Preprocessing

In [ ]:
# Text Preprocessing
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):


    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    html_pattern = r'<.*?>'
    text = re.sub(pattern=html_pattern, repl=' ', string=text)

    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(pattern=url_pattern, repl=' ', string=text)

    # numbers
    number_pattern = r'\d+'
    text = re.sub(pattern=number_pattern, repl=' ', string=text)

    # unidecode
    text = unidecode.unidecode(text)

    # Expanding Contractions
    text = contractions.fix(text)

    # remove punctutation
    text = text.translate(str.maketrans('', '', punctuation))

    # removing single characters
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    text = re.sub(pattern=single_char_pattern, repl=" ", string=text)

    # Extra spaces
    space_pattern = r'\s+'
    text = re.sub(pattern=space_pattern, repl=" ", string=text)

    # Tokenize and remove stopwords
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]

    # Join the words back into a single string
    text = ' '.join(words)

    return text
df['text'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# # Encoding the labels
import pandas as pd
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()  # Creates an instance of LabelEncoder.
df['class'] = label_encoder.fit_transform(df['class'])  # Encodes the 'class' column in the DataFrame using label encoding.


In [ ]:
# Split the DataFrame into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenizer for BERT model

In [ ]:
# Initialize BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(df['class'].unique()))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize and pad the training and testing data
train_tokens = tokenizer(list(train_df['text']), padding=True, truncation=True, return_tensors='pt', max_length=100)
test_tokens = tokenizer(list(test_df['text']), padding=True, truncation=True, return_tensors='pt', max_length=100)


In [ ]:
# Assuming train_tokens is a dictionary with keys 'input_ids', 'attention_mask', and 'token_type_ids'
top_5 = {
    'input_ids': train_tokens['input_ids'][0],
    'attention_mask': train_tokens['attention_mask'][0],
    'token_type_ids': train_tokens['token_type_ids'][0]
}

print(top_5)
print(top_5["token_type_ids"].size())


{'input_ids': tensor([  101,  2183,  3102,  2574,  2072,  2113,  2113,  2303,  2568,  3294,
        26351,  6427,  2183, 10797,  2172,  2130,  4167,  7210,  2145,  8239,
         5926,  3242,  8239, 19029,  3064,  3233,  3233,  7987,  5243,  2618,
         2117,  2936,  3233,  7567,  2303,  4167,  8239,  4301,  2342,  2619,
         3531,  2831,  2619,  3531,  2831,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 

In [ ]:
train_df.head()

,Unnamed: 0,text,class
103752,155958,going kill sooni know know body mind completel...,0
208014,312013,using sub diary day today great school stuff b...,1
220765,331077,else supposed got serious problem years back b...,0
116840,175538,hey yes need help get new subreddit ground sub...,1
149067,223708,non trans gender dysphoria like menstruation e...,1


In [ ]:
# Convert labels to PyTorch tensors
train_labels = torch.tensor(list(train_df['class']))
test_labels = torch.tensor(list(test_df['class']))

In [ ]:
# Create DataLoader for training and testing data
train_data = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], train_labels)
test_data = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'], test_labels)


In [ ]:
batch_size =32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
# Training loop (you can use your preferred training method)
optimizer = torch.optim.AdamW(bert_model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Model Implementation
Here we are running only 1 epoch due to constraints on hardware(GPU) resources

In [ ]:
# Train the model on your training data
num_epochs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)
for epoch in range(num_epochs):
    bert_model.train()
    total_loss = 0.0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = bert_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# Evaluation
bert_model.eval()

Epoch 1, Loss: 0.13095106706236703


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = bert_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Calculate classification metrics
print(classification_report(all_labels, all_preds))


              precision    recall  f1-score   support

           0       0.95      0.97      0.96     23128
           1       0.97      0.95      0.96     23287

    accuracy                           0.96     46415
   macro avg       0.96      0.96      0.96     46415
weighted avg       0.96      0.96      0.96     46415



# Saving the bert model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
bert_model.save_pretrained("/content/drive/MyDrive/Depression Detection/Bert_model")

# BERT IMPLEMENTATION on user input

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install transformers

In [ ]:
import nltk
import torch
from transformers import BertTokenizer, BertForSequenceClassification
nltk.download('punkt')
import re
!pip install autocorrect
from autocorrect import Speller
!pip install unidecode
import unidecode
!pip install contractions
import contractions
from string import punctuation

In [ ]:
# Text Preprocessing
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):


    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    html_pattern = r'<.*?>'
    text = re.sub(pattern=html_pattern, repl=' ', string=text)

    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(pattern=url_pattern, repl=' ', string=text)

    # numbers
    number_pattern = r'\d+'
    text = re.sub(pattern=number_pattern, repl=' ', string=text)

    # unidecode
    text = unidecode.unidecode(text)

    # Expanding Contractions
    text = contractions.fix(text)

    # remove punctutation
    text = text.translate(str.maketrans('', '', punctuation))

    # removing single characters
    single_char_pattern = r'\s+[a-zA-Z]\s+'
    text = re.sub(pattern=single_char_pattern, repl=" ", string=text)

    # Extra spaces
    space_pattern = r'\s+'
    text = re.sub(pattern=space_pattern, repl=" ", string=text)

    # Tokenize and remove stopwords
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]

    # Join the words back into a single string
    text = ' '.join(words)

    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Importing model and tokenizer

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# Define the path to the saved transformer model on Google Drive
model_path = '/content/drive/MyDrive/Depression Dataset/Bert_model'
# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
user_input = "I am feeling lonely and sad "
user_input=preprocess_text(user_input)

In [ ]:
# Initialize BERT model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Choose the appropriate device
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
# Move the model to the selected device
model.to(device)

# Tokenize the user input
user_tokens = tokenizer(user_input, padding=True, truncation=True, return_tensors='pt', max_length=100)

# Ensure that user_tokens are on the same device as the model
user_tokens = {key: value.to(device) for key, value in user_tokens.items()}

# Inference
with torch.no_grad():
    # Forward pass through the model
    outputs = model(input_ids=user_tokens['input_ids'], attention_mask=user_tokens['attention_mask'])
    logits = outputs.logits

# Interpret the model's output
predicted_class = torch.argmax(logits, dim=1).item()

# Get the corresponding label from the encoder
if predicted_class==0:
  print("depressed")
else:
  print("not depressed")


not depressed


**END**